In [ ]:
# libraries
import pandas as pd
import numpy as np
import os
from statsmodels.tsa.stattools import acf, adfuller
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.svm import SVR
from tqdm import tqdm
from collections import defaultdict
from sklearn import linear_model

%store -r Kelmarsh_df Penmanshiel_df

### pre-processing

In [ ]:
def model_preprocess(data, demand, temperature, n_lags):

    # Scale the input data
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data[[f'{demand}', f'{temperature}']])

    # Create lag features
    def create_lag_features(data, n_lags):
        lag_features = []
        for i in range(1, n_lags + 1):
            lag_features.append(data.shift(i).rename(columns=lambda x: f'{x}_lag_{i}'))
        return pd.concat(lag_features, axis=1)

    lag_features = create_lag_features(pd.DataFrame(scaled_data, columns=[f'{demand}', f'{temperature}']), n_lags)
    data = pd.DataFrame(scaled_data, columns=[f'{demand}', f'{temperature}']).join(lag_features).dropna().values

    # Train-test split (80:20)
    train_size = int(len(data) * 0.8)
    train, test = data[:train_size], data[train_size:]

    # Separate features and target variable
    X_train, y_train = train[:, 1:], train[:, 0]
    X_test, y_test = test[:, 1:], test[:, 0]

    # Reshape input
    X_train = X_train.reshape(X_train.shape[0], 1, -1)
    X_test = X_test.reshape(X_test.shape[0], 1, -1)

    # Convert to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float)
    y_train = torch.tensor(y_train, dtype=torch.float)
    X_test = torch.tensor(X_test, dtype=torch.float)
    y_test = torch.tensor(y_test, dtype=torch.float)

    return X_train, y_train, X_test, y_test, scaler

### data loader


In [ ]:
def model_dataloader(X_train, y_train, batch_size):
    train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float), torch.tensor(y_train, dtype=torch.float))
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size)

    return train_dataloader

### hyperparameter

In [ ]:
def model_hyperparameter(X_train):
    hyperparameters_dict = dict({
        'input_dim' : X_train.shape[2],
        'hidden_dim' : 60,
        'num_layers' : 1,
        'output_dim' : 1,
        'learning_rate' : 0.001,
        'num_epochs' : 5,
        'batch_size' : 32,
        'device' : "cpu",
        'nhead' : 4
    })

    return hyperparameters_dict

### errors

In [ ]:
def model_errors (results_df, model, y_true, y_pred):
    def mean_absolute_percentage_error(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)

    results_df.loc[f'{model}', 'MSE'] = '{:.6f}'.format(mse)
    results_df.loc[f'{model}', 'RMSE'] = '{:.6f}'.format(rmse)
    results_df.loc[f'{model}', 'MAE'] = '{:.6f}'.format(mae)
    results_df.loc[f'{model}', 'MAPE'] = '{:.6f}'.format(mape)

    return results_df

## Models 

### LSTM


In [ ]:
def model_LSTM(hyperparameters, dataloader, scaler, X_test, y_test):
    # Create the LSTM model
    class LSTMModel(nn.Module):
        def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
            super(LSTMModel, self).__init__()
            self.hidden_dim = hidden_dim
            self.num_layers = num_layers
            self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
            self.fc = nn.Linear(hidden_dim, output_dim)

        def forward(self, x):
            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim)#.to(hyperparameters['device'])
            c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim)#.to(hyperparameters['device'])
            with torch.backends.cudnn.flags(enabled=False):
                out, _ = self.lstm(x, (h0, c0))
            out = self.fc(out[:, -1, :])
            return out

    # Initialize the model, loss function, and optimizer
    model = LSTMModel(
        hyperparameters['input_dim'],
        hyperparameters['hidden_dim'],
        hyperparameters['num_layers'],
        hyperparameters['output_dim']
        ).to(hyperparameters['device'])
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=hyperparameters['learning_rate'])

    # Train the model
    model.train()
    for epoch in range(hyperparameters['num_epochs']):
        for x_batch, y_batch in dataloader:
            # move to GPU
            x_batch = x_batch.to(hyperparameters['device'])
            y_batch = y_batch.to(hyperparameters['device'])

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            y_pred = model(x_batch)

            # Calculate the loss
            loss = criterion(y_pred.squeeze(), y_batch)

            # Backward pass
            loss.backward()

            # Update the weights
            optimizer.step()

    # Make predictions
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test).detach().numpy().squeeze()

    # Invert scaling for test data
    test_unscaled = np.column_stack((y_test.numpy().reshape(-1, 1), X_test.numpy().squeeze()[:, :1]))
    test_unscaled = scaler.inverse_transform(test_unscaled)
    y_test_unscaled = test_unscaled[:, 0]

    # Invert scaling for predictions
    y_pred_scaled = np.column_stack((y_pred.reshape(-1, 1), X_test.numpy().squeeze()[:, :1]))
    y_pred_unscaled = scaler.inverse_transform(y_pred_scaled)[:, 0]

    return y_pred_unscaled, y_test_unscaled

### Transformer (ChatGPT)

In [ ]:
def model_Transformer(hyperparameters, dataloader, scaler, X_test, y_test):
    # Transformer Model
    class TransformerModel(nn.Module):
        def __init__(self, input_dim, d_model, nhead, num_layers, output_dim):
            super(TransformerModel, self).__init__()
            self.embedding = nn.Linear(input_dim, d_model)
            self.transformer_layer = nn.TransformerEncoderLayer(d_model, nhead)
            self.transformer = nn.TransformerEncoder(self.transformer_layer, num_layers)
            self.fc = nn.Linear(d_model, output_dim)

        def forward(self, x):
            x = self.embedding(x)
            x = self.transformer(x)
            x = self.fc(x)
            return x

    # Create the transformer model
    model = TransformerModel(
                input_dim = hyperparameters['input_dim'],
                d_model = hyperparameters['hidden_dim'],
                nhead = hyperparameters['nhead'],
                num_layers = hyperparameters['num_layers'],
                output_dim = hyperparameters['output_dim']
                ).to(hyperparameters['device'])

    # Loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=hyperparameters['learning_rate'])

    # Training loop
    for epoch in range(hyperparameters['num_epochs']):
        for x_batch, y_batch in dataloader:
            # Move data to the device (CPU or GPU)
            x_batch = x_batch.to(hyperparameters['device'])
            y_batch = y_batch.to(hyperparameters['device'])

            #print(y_batch.shape)
            #print(x_batch.shape)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            y_pred = model(x_batch)

            #print(y_pred.shape)

            # Calculate the loss
            loss = criterion(y_pred.squeeze(), y_batch)

            # Backward pass
            loss.backward()

            # Update the weights
            optimizer.step()

    # Make predictions
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test).detach().cpu().numpy().squeeze()

    # Invert scaling for test data
    test_unscaled = np.column_stack((y_test.cpu().numpy().reshape(-1, 1), X_test.cpu().numpy().squeeze()[:, :1]))
    test_unscaled = scaler.inverse_transform(test_unscaled)
    y_test_unscaled = test_unscaled[:, 0]

    # Invert scaling for predictions
    y_pred_scaled = np.column_stack((y_pred.reshape(-1, 1), X_test.cpu().numpy().squeeze()[:, :1]))
    y_pred_unscaled = scaler.inverse_transform(y_pred_scaled)[:, 0]

    return y_pred_unscaled, y_test_unscaled

## Compelete Function

In [ ]:
def final_model(data, demand, temperature, n_lags):
    # create results df
    results_df = pd.DataFrame(
        index=['linear regression', 'LSTM', 'Transformer 1'],
        columns=['MSE', 'RMSE', 'MAE', 'MAPE']
        )

    # preprocess data
    X_train, y_train, X_test, y_test, scaler = model_preprocess(data, demand, temperature, n_lags)
    linear_x_train, linear_x_test, linear_y_train, linear_y_test = model_linear_data(data, demand, temperature, n_lags)

    # hyperparameters
    hyperparameters = model_hyperparameter(X_train)

    # dataloader
    dataloader = model_dataloader(X_train, y_train, hyperparameters['batch_size'])

    # predictions
    linear_pred = model_linear(linear_x_train, linear_y_train, linear_x_test)
    LSTM_pred, LSTM_test = model_LSTM(hyperparameters, dataloader, scaler, X_test, y_test)
    #Transformer_pred, Transformer_test = model_Transformer(hyperparameters, dataloader, scaler, X_test, y_test)

    # errors
    model_errors(results_df, 'linear regression', linear_y_test, linear_pred)
    model_errors(results_df, 'LSTM', LSTM_test, LSTM_pred)
    #model_errors(results_df, 'Transformer 1', Transformer_test, Transformer_pred)

    return results_df

In [ ]:
final_model(Kelmarsh_1, 'Energy Export (kWh)', 'Long Term Wind (m/s)', 24)